In [1]:
import numpy as np
import scipy as sp

In [39]:
np.ones((7,1))
np.arange(7)+1

array([1, 2, 3, 4, 5, 6, 7])

In [33]:
random_matrix = np.random.normal(size=(2,3))
random_matrix

array([[ 1.00600146,  1.30606621, -1.28630933],
       [-0.49630937,  1.41270871, -0.7851829 ]])

In [34]:
random_matrix.cumsum(axis=1)

array([[ 1.00600146,  2.31206767,  1.02575834],
       [-0.49630937,  0.91639934,  0.13121644]])

In [26]:
c = np.ones(7)*2
A = np.random.randint(5,size=(7,7))
A

array([[0, 0, 2, 0, 0, 0, 1],
       [0, 1, 4, 1, 4, 2, 4],
       [1, 4, 1, 0, 3, 4, 2],
       [4, 4, 3, 4, 1, 0, 2],
       [4, 2, 4, 1, 1, 3, 3],
       [0, 1, 2, 2, 1, 1, 4],
       [0, 0, 0, 1, 4, 0, 1]])

In [29]:
A[:,1]=c
A

array([[0, 2, 2, 0, 0, 0, 1],
       [0, 2, 4, 1, 4, 2, 4],
       [1, 2, 1, 0, 3, 4, 2],
       [4, 2, 3, 4, 1, 0, 2],
       [4, 2, 4, 1, 1, 3, 3],
       [0, 2, 2, 2, 1, 1, 4],
       [0, 2, 0, 1, 4, 0, 1]])

In [43]:
A+np.expand_dims(np.arange(1,8),1)+10

array([[11, 13, 13, 11, 11, 11, 12],
       [12, 14, 16, 13, 16, 14, 16],
       [14, 15, 14, 13, 16, 17, 15],
       [18, 16, 17, 18, 15, 14, 16],
       [19, 17, 19, 16, 16, 18, 18],
       [16, 18, 18, 18, 17, 17, 20],
       [17, 19, 17, 18, 21, 17, 18]])

$$dS = r S dt + \sigma S dW_t$$
$$log S_{t+1} = log S_t +(r - \frac{\sigma^2}{2})\Delta t + \sigma \sqrt{\Delta t} \epsilon$$
where
    $$\epsilon \sim N(0,1)$$


In [49]:
class MonteCarlo:
    def __init__(self,S0,K,T,r,sigma,underlying_process="geometric brownian motion"):
        self.underlying_process = underlying_process
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        
    def simulate(self, n_trails, n_steps):
        dt = self.T/n_steps
        if(self.underlying_process=="geometric brownian motion"):
#             first_step_prices = np.ones((n_trails,1))*np.log(self.S0)
            log_price_matrix = np.zeros((n_trails,n_steps))
            normal_matrix = np.random.normal(size=(n_trails,n_steps))
            cumsum_normal_matrix = normal_matrix.cumsum(axis=1)
#             log_price_matrix = np.concatenate((first_step_prices,log_price_matrix),axis=1)
            deviation_matrix = cumsum_normal_matrix*self.sigma*np.sqrt(dt) + \
    (self.r-self.sigma**2/2)*dt*np.arange(1,n_steps+1)
            log_price_matrix = deviation_matrix+np.log(self.S0)
        
        return log_price_matrix

In [50]:
mc = MonteCarlo(10,10,1,0.02,0.3)

In [53]:
log_price_matrix = mc.simulate(10,100)

In [62]:
funcs = [lambda x: x**0, lambda x: x, lambda x: x**2]
np.array([func(np.array([3,2,4,5,6])) for func in funcs]).T

array([[ 1,  3,  9],
       [ 1,  2,  4],
       [ 1,  4, 16],
       [ 1,  5, 25],
       [ 1,  6, 36]])

In [65]:
A*np.array([1,2,3,4,5,6,7])

array([[ 0,  4,  6,  0,  0,  0,  7],
       [ 0,  4, 12,  4, 20, 12, 28],
       [ 1,  4,  3,  0, 15, 24, 14],
       [ 4,  4,  9, 16,  5,  0, 14],
       [ 4,  4, 12,  4,  5, 18, 21],
       [ 0,  4,  6,  8,  5,  6, 28],
       [ 0,  4,  0,  4, 20,  0,  7]])

In [66]:
def calculate_Q_matrix(log_price_matrix,r,dt,step_k):
    func_list = [lambda x: x**0, lambda x: x, lambda x: x**2]
    n_trails,n_steps = log_price_matrix.shape
    S_k = np.exp(log_price_matrix[:,step_k-1])
    S_kp1 = np.exp(log_price_matrix[:,step_k])
    DS = np.exp(-r*dt)*S_kp1 - S_k
    A = np.array([func(S_k) for func in func_list]).T
    B = (np.array([func(S_k) for func in func_list])*DS).T
    return np.concatenate((-A,B),axis=1)

array([[-1.00000000e+00, -9.15790484e+00, -8.38672210e+01,
        -2.56531772e-01, -2.34929355e+00, -2.15146068e+01],
       [-1.00000000e+00, -9.67293515e+00, -9.35656744e+01,
         1.17183369e-01,  1.13350712e+00,  1.09643409e+01],
       [-1.00000000e+00, -1.19233828e+01, -1.42167057e+02,
        -2.16745656e-01, -2.58434142e+00, -3.08140919e+01],
       [-1.00000000e+00, -9.08478928e+00, -8.25333963e+01,
         1.94703919e-01,  1.76884407e+00,  1.60695757e+01],
       [-1.00000000e+00, -9.64688107e+00, -9.30623144e+01,
        -8.36758663e-02, -8.07211131e-01, -7.78706978e+00],
       [-1.00000000e+00, -8.93009603e+00, -7.97466151e+01,
        -8.28285040e-02, -7.39666495e-01, -6.60529283e+00],
       [-1.00000000e+00, -1.11720814e+01, -1.24815403e+02,
        -2.18108177e-01, -2.43672232e+00, -2.72232601e+01],
       [-1.00000000e+00, -1.18236112e+01, -1.39797783e+02,
        -3.45416062e-01, -4.08406522e+00, -4.82883994e+01],
       [-1.00000000e+00, -1.06192767e+01, -1.127